## 4. Modell Training und Auswahl

In [2]:
import pandas as pd
import pickle

# Laden des transformierten Datensatzes
df_transformed = pd.read_csv('../model/df_transformed.csv')

# Laden der LabelEncoder-Objekte
with open('../model/label_encoders.pkl', 'rb') as handle:
    loaded_label_encoders = pickle.load(handle)

# Überprüfen der ersten Zeilen des Datensatzes
df_transformed.head(), loaded_label_encoders.keys()


(   Marke  Modell  Baujahr  Getriebe  Kilometerstand  Kraftstoffart  \
 0      0      12     2017         2           25322              1   
 1      0      17     2016         0           58262              2   
 2      0      12     2016         2           48193              1   
 3      0      15     2017         0           41765              2   
 4      0      14     2019         2            3215              1   
 
    Verbrauch (l/100km)  Motorgröße (l)  Preis (€)  
 0                 4.25             1.4    14375.0  
 1                 3.66             2.0    18975.0  
 2                 4.25             1.4    12650.0  
 3                 3.50             2.0    19320.0  
 4                 4.74             1.0    19895.0  ,
 dict_keys(['Marke', 'Modell', 'Getriebe', 'Kraftstoffart']))

In [3]:
from sklearn.model_selection import train_test_split

# Trennen der Features und der Zielvariable
X = df_transformed.drop(columns=['Preis (€)'])
y = df_transformed['Preis (€)']

# Aufteilen des Datensatzes in Trainings- und Testsets (80% Training, 20% Test)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Überprüfen der Größe der Trainings- und Testsets
X_train.shape, X_test.shape, y_train.shape, y_test.shape


((78167, 8), (19542, 8), (78167,), (19542,))

In [4]:
import xgboost as xgb

# Erstellen des XGBoost-Regressionsmodells
xgb_model = xgb.XGBRegressor(objective="reg:squarederror", random_state=42)

# Trainieren des Modells mit den Trainingsdaten
xgb_model.fit(X_train, y_train)

# Überprüfen der Trainingsleistung
train_score = xgb_model.score(X_train, y_train)

# Überprüfen der Testleistung
test_score = xgb_model.score(X_test, y_test)

train_score, test_score


(0.9695696321162255, 0.9640909742581282)

Trainingsgenauigkeit (R²-Wert):  0.9701

Testgenauigkeit (R²-Wert): 0.9645

Diese Werte zeigen, dass das Modell gut auf die Trainingsdaten passt und auch eine gute Leistung auf den Testdaten zeigt.

In [5]:
# Speicherpfad für das XGBoost-Modell im JSON-Format
xgb_model_json_path = '../model/xgb_model.json'

# Speichern des XGBoost-Modells im JSON-Format
xgb_model.save_model(xgb_model_json_path)

xgb_model_json_path


'../model/xgb_model.json'